## 1. 对GPS data进行的整理
     * 合并所有gps文件夹里的数据
     * 将UTC time转换成local time（东部时间）

In [1]:
import pandas as pd

In [2]:
#combine all data in gps directory
#合并gps文件夹里的所有数据
import os
filelst=os.listdir("./gps")
combined_csv = pd.concat([pd.read_csv("gps/"+f) for f in filelst])
combined_csv.head(10)

,timestamp,UTC time,latitude,longitude,altitude,accuracy
0,1570812580322,2019-10-11T16:49:40.322,40.513542,-79.804057,255.351288,65.0
1,1570812580906,2019-10-11T16:49:40.906,40.513547,-79.804057,255.318161,65.0
2,1570812580906,2019-10-11T16:49:40.906,40.513547,-79.804057,255.309921,65.0
3,1570812585999,2019-10-11T16:49:45.999,40.513774,-79.804194,258.231655,50.0
4,1570812586999,2019-10-11T16:49:46.999,40.513716,-79.804134,257.100918,50.0
5,1570812587999,2019-10-11T16:49:47.999,40.513661,-79.804138,257.500637,50.0
6,1570812588999,2019-10-11T16:49:48.999,40.513713,-79.804135,257.714382,30.0
7,1570812589999,2019-10-11T16:49:49.999,40.513713,-79.804135,257.679165,30.0
8,1570812590999,2019-10-11T16:49:50.999,40.513713,-79.804135,257.692837,30.0
9,1570812591999,2019-10-11T16:49:51.999,40.513571,-79.804065,257.641689,10.0


In [3]:
#importing both the modules
import datetime
import pytz

In [4]:
# 去掉“UTC time”那列中间的“T”
combined_csv["UTC time"]=combined_csv["UTC time"].apply(lambda x: x[:10]+" "+x[11:19])
# 将 “UTC time"那列转换成date
combined_csv["UTC time"]=combined_csv["UTC time"].astype("datetime64[ns]")
combined_csv=combined_csv.reset_index()

In [5]:
#function to convert UTC time to Eastern time
# 将UTC time 转换成美东时间
# ref: https://www.askpython.com/python/examples/utc-to-est-conversion#:~:text=Converting%20UTC%20to%20EST%20in%20Python%20can%20be%20done%20using,time%2C%20making%20the%20conversion%20accurate.
def convert_utc_to_est(utc_time):
  utc_time=utc_time.tz_localize(tz = 'UTC')
  #fetch the timezone information
  est = pytz.timezone('US/Eastern')
 
  #convert utc to est
  est_time = utc_time.astimezone(est)
 
  #return the converted value
  return est_time

combined_csv["local_time"]=combined_csv["UTC time"].apply(lambda x: convert_utc_to_est(x))
combined_csv.head(10)

,index,timestamp,UTC time,latitude,longitude,altitude,accuracy,local_time
0,0,1570812580322,2019-10-11 16:49:40,40.513542,-79.804057,255.351288,65.0,2019-10-11 12:49:40-04:00
1,1,1570812580906,2019-10-11 16:49:40,40.513547,-79.804057,255.318161,65.0,2019-10-11 12:49:40-04:00
2,2,1570812580906,2019-10-11 16:49:40,40.513547,-79.804057,255.309921,65.0,2019-10-11 12:49:40-04:00
3,3,1570812585999,2019-10-11 16:49:45,40.513774,-79.804194,258.231655,50.0,2019-10-11 12:49:45-04:00
4,4,1570812586999,2019-10-11 16:49:46,40.513716,-79.804134,257.100918,50.0,2019-10-11 12:49:46-04:00
5,5,1570812587999,2019-10-11 16:49:47,40.513661,-79.804138,257.500637,50.0,2019-10-11 12:49:47-04:00
6,6,1570812588999,2019-10-11 16:49:48,40.513713,-79.804135,257.714382,30.0,2019-10-11 12:49:48-04:00
7,7,1570812589999,2019-10-11 16:49:49,40.513713,-79.804135,257.679165,30.0,2019-10-11 12:49:49-04:00
8,8,1570812590999,2019-10-11 16:49:50,40.513713,-79.804135,257.692837,30.0,2019-10-11 12:49:50-04:00
9,9,1570812591999,2019-10-11 16:49:51,40.513571,-79.804065,257.641689,10.0,2019-10-11 12:49:51-04:00


## 2. 对ubertrips data进行的整理
        * 选择123用户 
        * 统一坐上uber时间为r_time
        * 统一下车时间为d_time

In [7]:
#读取并筛选123用户
ut=pd.read_csv("UberTrips.csv")
ut=ut[ut["UserID"]=="SG00123"]
#这边仅筛选了一些比较重要的列 比如上下车时间 距离
ut=ut[['Request Date (Local)','Request Time (Local)','Drop-off Date (Local)',
       'Drop-off Time (Local)','Pickup Address',
       'Drop-off Address','Distance (mi)']]

In [8]:
#function: 统一上下车时间的函数
def time_convert(x):
    if x[-2]=="A":
        return x[:-2]
    elif x[:-5]=="12":
        return x[:-2]
    else:
        return str(int(x[:-5])+12)+x[-5:-2]
#function：将时间改成东部时间
def loclize(x):
    return x.tz_localize('US/Eastern')

# 运用function到uber的数据库上
ut['Request Time (Local)']=ut['Request Time (Local)'].apply(lambda x: time_convert(x))
ut['Drop-off Time (Local)']=ut['Drop-off Time (Local)'].apply(lambda x: time_convert(x))
ut["r_time"]=ut['Request Date (Local)']+" "+ut['Request Time (Local)']
ut["d_time"]=ut['Drop-off Date (Local)']+" "+ut["Drop-off Time (Local)"]
ut[['r_time','d_time']]=ut[['r_time','d_time']].astype("datetime64[ns]")
ut['r_time']=ut['r_time'].apply(lambda x: loclize(x))
ut['d_time']=ut['d_time'].apply(lambda x: loclize(x))


ut=ut.reset_index()

In [9]:
ut.head(10)

,index,Request Date (Local),Request Time (Local),Drop-off Date (Local),Drop-off Time (Local),Pickup Address,Drop-off Address,Distance (mi),r_time,d_time
0,73,10/18/2019,18:06,10/18/2019,18:38,"Penn Hills, PA 15147, USA","2403 Freeport Rd, Pittsburgh, PA 15238, USA",3.62,2019-10-18 18:06:00-04:00,2019-10-18 18:38:00-04:00
1,74,10/18/2019,18:46,10/18/2019,19:00,"2403 Freeport Rd, Pittsburgh, PA 15238, USA","Penn Hills, PA 15147, USA",3.6,2019-10-18 18:46:00-04:00,2019-10-18 19:00:00-04:00
2,75,10/19/2019,18:26,10/19/2019,18:36,"Penn Hills, PA 15147, USA","Plum, PA 15147, USA",1.41,2019-10-19 18:26:00-04:00,2019-10-19 18:36:00-04:00
3,76,10/19/2019,18:39,10/19/2019,18:46,"Plum, PA 15147, USA","Penn Hills, PA 15147, USA",1.33,2019-10-19 18:39:00-04:00,2019-10-19 18:46:00-04:00
4,77,11/3/2019,11:37,11/3/2019,12:09,"Penn Hills, PA 15147, USA","1610 Old Leechburg Rd, New Kensington, PA 1506...",7.73,2019-11-03 11:37:00-05:00,2019-11-03 12:09:00-05:00
5,78,11/3/2019,21:30,11/3/2019,21:50,"1609 Old Leechburg Rd, New Kensington, PA 1506...","Penn Hills, PA 15147, USA",7.73,2019-11-03 21:30:00-05:00,2019-11-03 21:50:00-05:00
6,79,11/4/2019,18:22,11/4/2019,18:49,"Penn Hills, PA 15147, USA","Plum, PA 15147, USA",1.46,2019-11-04 18:22:00-05:00,2019-11-04 18:49:00-05:00
7,80,11/4/2019,18:54,11/4/2019,19:04,"Plum, PA 15147, USA","Penn Hills, PA 15147, USA",1.32,2019-11-04 18:54:00-05:00,2019-11-04 19:04:00-05:00
8,81,11/5/2019,18:14,11/5/2019,18:55,"610612 Beatty Rd, Monroeville, PA 15146, USA","Penn Hills, PA 15147, USA",11.49,2019-11-05 18:14:00-05:00,2019-11-05 18:55:00-05:00
9,82,11/5/2019,20:49,11/5/2019,21:15,"Penn Hills, PA 15147, USA","1609 Old Leechburg Rd, New Kensington, PA 1506...",7.74,2019-11-05 20:49:00-05:00,2019-11-05 21:15:00-05:00


## 3. 找到每条uber trip同一时间本人所在的位置（gps data)

In [11]:
#helper function
def find(rawnumber):
    return combined_csv[combined_csv['local_time'].between(ut['r_time'][rawnumber-1], ut['d_time'][rawnumber-1])]

In [14]:
# 找到uber第一行trip对应统一时间该用户的gps data
fr=find(1)
fr.head(10)

,index,timestamp,UTC time,latitude,longitude,altitude,accuracy,local_time
41732,0,1571436499000,2019-10-18 22:08:19,40.513568,-79.804355,255.696255,10.0,2019-10-18 18:08:19-04:00
41733,1,1571436507000,2019-10-18 22:08:27,40.513568,-79.804355,255.696255,10.0,2019-10-18 18:08:27-04:00
41734,2,1571436534763,2019-10-18 22:08:54,40.513568,-79.804355,255.696259,65.0,2019-10-18 18:08:54-04:00
41735,3,1571436535053,2019-10-18 22:08:55,40.513471,-79.804259,255.645599,65.0,2019-10-18 18:08:55-04:00
41736,4,1571436536442,2019-10-18 22:08:56,40.513568,-79.804355,256.008389,5.0,2019-10-18 18:08:56-04:00
41737,5,1571436537000,2019-10-18 22:08:57,40.513568,-79.804355,255.954678,5.0,2019-10-18 18:08:57-04:00
41738,6,1571436538000,2019-10-18 22:08:58,40.513568,-79.804355,255.685390,5.0,2019-10-18 18:08:58-04:00
41739,7,1571436539000,2019-10-18 22:08:59,40.513568,-79.804355,255.596157,5.0,2019-10-18 18:08:59-04:00
41740,8,1571436540000,2019-10-18 22:09:00,40.513568,-79.804355,255.589687,5.0,2019-10-18 18:09:00-04:00
41741,9,1571436541000,2019-10-18 22:09:01,40.513568,-79.804355,255.583523,5.0,2019-10-18 18:09:01-04:00


In [15]:
# 找到uber第二行trip对应统一时间该用户的gps data
sr=find(2)
sr.head(10)

,index,timestamp,UTC time,latitude,longitude,altitude,accuracy,local_time
41970,238,1571439208000,2019-10-18 22:53:28,40.513766,-79.804130,254.070827,30.000000,2019-10-18 18:53:28-04:00
41971,239,1571439252390,2019-10-18 22:54:12,40.513378,-79.804179,255.471781,20.000000,2019-10-18 18:54:12-04:00
41972,240,1571439415507,2019-10-18 22:56:55,40.513836,-79.804153,254.070831,134.318533,2019-10-18 18:56:55-04:00
41973,241,1571439415782,2019-10-18 22:56:55,40.513368,-79.804218,255.466156,65.000000,2019-10-18 18:56:55-04:00
41974,242,1571439416977,2019-10-18 22:56:56,40.513380,-79.804218,255.306564,65.000000,2019-10-18 18:56:56-04:00
41975,243,1571439422999,2019-10-18 22:57:02,40.513383,-79.804218,256.653347,50.000000,2019-10-18 18:57:02-04:00
41976,244,1571439423999,2019-10-18 22:57:03,40.513383,-79.804218,256.622402,30.000000,2019-10-18 18:57:03-04:00
41977,245,1571439424999,2019-10-18 22:57:04,40.513383,-79.804218,256.578701,30.000000,2019-10-18 18:57:04-04:00
41978,246,1571439425999,2019-10-18 22:57:05,40.513383,-79.804218,256.553310,10.000000,2019-10-18 18:57:05-04:00
41979,247,1571439426999,2019-10-18 22:57:06,40.513383,-79.804218,256.547573,10.000000,2019-10-18 18:57:06-04:00
